In [1]:
#加载模块
import time
import copy
import os
from PIL import ImageFile
import torch
import torchvision
ImageFile.LOAD_TRUNCATED_IMAGES = True
from fine_tuning_config_file import *
from sklearn.metrics import auc
from sklearn.metrics import roc_auc_score, classification_report
import numpy as np

In [2]:
#采用gpu训练模型
use_gpu = GPU_MODE
if use_gpu:
    torch.cuda.set_device(CUDA_DEVICE)

In [3]:
#数据转换，为了扩增数据
data_transforms = {
    'train': torchvision.transforms.Compose([
        torchvision.transforms.RandomResizedCrop(224),
        torchvision.transforms.RandomHorizontalFlip(),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': torchvision.transforms.Compose([
        torchvision.transforms.Resize(224),
        torchvision.transforms.CenterCrop(224),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}
#加载数据集
data_dir = DATA_DIR
dsets = {x: torchvision.datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
         for x in ['train', 'val']}
dset_loaders = {x: torch.utils.data.DataLoader(dsets[x], batch_size=BATCH_SIZE,
                                               shuffle=True, num_workers=25)
                for x in ['train', 'val']}
dset_sizes = {x: len(dsets[x]) for x in ['train', 'val']}
dset_classes = dsets['train'].classes

In [4]:
#数据增强
def mixup_data(x, y, alpha=1.0, use_cuda=True):
    '''Returns mixed inputs, pairs of targets, and lambda'''
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = x.size()[0]
    if use_cuda:
        index = torch.randperm(batch_size).cuda()
    else:
        index = torch.randperm(batch_size)

    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam


def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)


In [5]:
#训练网络模型
def train_model(model, criterion, optimizer, lr_scheduler, num_epochs=100):
    since = time.time()

    best_model = model
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                mode='train'
                optimizer = lr_scheduler(optimizer, epoch)
                model.train()  # Set model to training mode
            else:
                model.eval()
                mode='val'

            train_loss = 0
            reg_loss = 0
            correct = 0
            total = 0

            counter=0
            # Iterate over data.
            for batch_idx, (inputs, targets) in enumerate(dset_loaders[phase]):
                inputs, targets = inputs.cuda(), targets.cuda()

                inputs, targets_a, targets_b, lam = mixup_data(inputs, targets, 0.2, use_cuda=True)
                inputs, targets_a, targets_b = map(Variable, (inputs,targets_a, targets_b))
                outputs = model(inputs)
                loss = mixup_criterion(criterion, outputs, targets_a, targets_b, lam)
                train_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                if counter==0:
                    phase_pred=predicted.cpu().numpy()
                    phase_label_a=targets_a.cpu().numpy()
                    phase_label_b=targets_b.cpu().numpy()
                else:
                    phase_pred = np.append(phase_pred, predicted.cpu().numpy())
                    phase_label_a = np.append(phase_label_a,targets_a.cpu().numpy())
                    phase_label_b = np.append(phase_label_b,targets_b.cpu().numpy())
                total += targets.size(0)
                correct += (lam * predicted.eq(targets_a.data).cpu().sum().float()
                    + (1 - lam) * predicted.eq(targets_b.data).cpu().sum().float())
                counter+=1
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            epoch_loss = train_loss/(batch_idx+1)
            epoch_acc = 100.*correct/total
            epoch_auc = (lam*roc_auc_score(phase_label_a, phase_pred)+(1-lam)*roc_auc_score(phase_label_b, phase_pred))
            print('{} Loss: {:.4f} Acc: {:.4f} Auc: {:.4f} '.format(
                phase, epoch_loss,epoch_acc,epoch_auc))


            # deep copy the model
            if phase == 'val':
                """if USE_TENSORBOARD:
                    foo.add_scalar_value('epoch_loss',epoch_loss,step=epoch)
                    foo.add_scalar_value('epoch_acc',epoch_acc,step=epoch)"""
                if epoch_acc > best_acc:
                    best_acc = epoch_acc
                    best_model = copy.deepcopy(model)
                    print('new best accuracy = ',best_acc)
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    print('returning and looping back')
    return best_model

In [6]:
#改变训练模型的学习率
def exp_lr_scheduler(optimizer, epoch, init_lr=BASE_LR, lr_decay_epoch=EPOCH_DECAY):
    """Decay learning rate by a factor of DECAY_WEIGHT every lr_decay_epoch epochs."""
    lr = init_lr * (DECAY_WEIGHT**(epoch // lr_decay_epoch))

    if epoch % lr_decay_epoch == 0:
        print('LR is set to {}'.format(lr))

    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

    return optimizer

In [7]:
#用迁移学习搭建自己的网络
class BCNN(torch.nn.Module):
    """B-CNN for CUB200.

    The B-CNN model is illustrated as follows.
    conv1^2 (64) -> pool1 -> conv2^2 (128) -> pool2 -> conv3^3 (256) -> pool3
    -> conv4^3 (512) -> pool4 -> conv5^3 (512) -> bilinear pooling
    -> sqrt-normalize -> L2-normalize -> fc (200).
    The network accepts a 3*448*448 input, and the pool5 activation has shape
    512*28*28 since we down-sample 5 times.
    vgg16：conv1^2 (64) -> pool1 -> conv2^2 (128) -> pool2 -> conv3^3 (256) -> pool3
    -> conv4^3 (512) -> pool4 -> conv5^3 (512) 
    Attributes:
        features, torch.nn.Module: Convolution and pooling layers.
        fc, torch.nn.Module: 200.
    """
    def __init__(self):
        """Declare all needed layers."""
        torch.nn.Module.__init__(self)
        # Convolution and pooling layers of VGG-16.
        self.features = torchvision.models.vgg16(pretrained=True,progress=True).features#获取vgg16的特征
        self.features = torch.nn.Sequential(*list(self.features.children())#获取零到最后第二层的特征
                                            [:-1])  # Remove pool5.
        # Linear classifier.
        self.fc = torch.nn.Linear(512**2, 2)#定义一个输入特征为512*2，输出特征为200的线性层
        for param in self.features.parameters():
            param.requires_grad = False


    def forward(self, X):
        """Forward pass of the network.

        Args:
            X, torch.autograd.Variable of shape N*3*448*448.

        Returns:
            Score, torch.autograd.Variable of shape N*200.
        """
        N = X.size()[0]#n为x的第一维形状
        assert X.size() == (N, 3, 224, 224)#判断x的形状是否为（n，3，448，448）
        X = self.features(X)#？？
        assert X.size() == (N, 512, 14, 14)#判断x的形状是否为（n，512，28，28）
        X = X.view(N, 512, 14**2)#转换x的形状为(n，512，28*28)
        X = torch.bmm(X, torch.transpose(X, 1, 2)) / (14**2)  # Bilinear  x矩阵乘x的转置然后除28*28
        assert X.size() == (N, 512, 512)#判断x的形状是否为
        X = X.view(N, 512**2)
        X = torch.sqrt(X + 1e-5)#开根号
        X = torch.nn.functional.normalize(X)#进行参数规范化
        X = self.fc(X)#加入线性层，输出为（n，200）
        assert X.size() == (N, 2)
        return X

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
    
class FocalLoss(nn.Module):
    def __init__(self, gamma=0.25, alpha=None, size_average=True):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        if isinstance(alpha,(float,int)): self.alpha = torch.Tensor([alpha,1-alpha])
        if isinstance(alpha,list): self.alpha = torch.Tensor(alpha)
        self.size_average = size_average

    def forward(self, input, target):
        if input.dim()>2:
            input = input.view(input.size(0),input.size(1),-1)  # N,C,H,W => N,C,H*W
            input = input.transpose(1,2)    # N,C,H*W => N,H*W,C
            input = input.contiguous().view(-1,input.size(2))   # N,H*W,C => N*H*W,C
        target = target.view(-1,1)

        logpt = F.log_softmax(input)
        logpt = logpt.gather(1,target)
        logpt = logpt.view(-1)
        pt = Variable(logpt.data.exp())

        if self.alpha is not None:
            if self.alpha.type()!=input.data.type():
                self.alpha = self.alpha.type_as(input.data)
            at = self.alpha.gather(0,target.data.view(-1))
            logpt = logpt * Variable(at)

        loss = -1 * (1-pt)**self.gamma * logpt
        if self.size_average: return loss.mean()
        else: return loss.sum()

In [ ]:
model_ft = torch.nn.DataParallel(BCNN()).cuda()

criterion = torch.nn.CrossEntropyLoss()
#criterion = FocalLoss()

if use_gpu:
    criterion.cuda()
    model_ft = model_ft.cuda()

optimizer_ft = torch.optim.RMSprop(model_ft.parameters(), lr=0.0001)


# Run the functions and save the best model in the function model_ft.
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=30)

# Save model
#model_ft.save_state_dict('fine_tuned_best_model.pt')
torch.save(model_ft.state_dict())

Epoch 0/29
----------
LR is set to 0.001
train Loss: 0.5209 Acc: 74.3706 Auc: 0.6508 
val Loss: 0.4086 Acc: 82.0247 Auc: 0.6914 
new best accuracy =  tensor(82.0247)
Epoch 1/29
----------
train Loss: 0.4947 Acc: 76.2022 Auc: 0.6608 
val Loss: 0.3711 Acc: 84.4089 Auc: 0.6973 
new best accuracy =  tensor(84.4089)
Epoch 2/29
----------
train Loss: 0.4862 Acc: 76.9466 Auc: 0.6675 
val Loss: 0.3508 Acc: 86.2379 Auc: 0.7251 
new best accuracy =  tensor(86.2379)
Epoch 3/29
----------
train Loss: 0.4763 Acc: 77.4599 Auc: 0.6699 
val Loss: 0.3386 Acc: 86.4548 Auc: 0.7226 
new best accuracy =  tensor(86.4548)
Epoch 4/29
----------
train Loss: 0.4778 Acc: 77.2199 Auc: 0.6654 
val Loss: 0.3279 Acc: 87.8362 Auc: 0.7284 
new best accuracy =  tensor(87.8362)
Epoch 5/29
----------
train Loss: 0.4683 Acc: 78.0591 Auc: 0.6717 
val Loss: 0.3119 Acc: 88.2741 Auc: 0.7383 
new best accuracy =  tensor(88.2741)
Epoch 6/29
----------
train Loss: 0.4661 Acc: 78.1618 Auc: 0.6730 
val Loss: 0.3113 Acc: 88.6732 Au

In [ ]:
model_ft = torch.nn.DataParallel(BCNN()).cuda()

criterion = torch.nn.CrossEntropyLoss()
#criterion = FocalLoss(gamma=0)

if use_gpu:
    criterion.cuda()
    model_ft = model_ft.cuda()

optimizer_ft = torch.optim.RMSprop(model_ft.parameters(), lr=0.0001)


# Run the functions and save the best model in the function model_ft.
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=30)

# Save model
#model_ft.save_state_dict('fine_tuned_best_model.pt')
torch.save(model_ft.state_dict())

Epoch 0/29
----------
LR is set to 0.001
train Loss: 0.5210 Acc: 74.5567 Auc: 0.6474 
val Loss: 0.4041 Acc: 82.3743 Auc: 0.7028 
new best accuracy =  tensor(82.3743)
Epoch 1/29
----------
train Loss: 0.4937 Acc: 76.4639 Auc: 0.6582 
val Loss: 0.3829 Acc: 84.1254 Auc: 0.7029 
new best accuracy =  tensor(84.1254)
Epoch 2/29
----------
train Loss: 0.4851 Acc: 76.8511 Auc: 0.6621 
val Loss: 0.3543 Acc: 85.6500 Auc: 0.7262 
new best accuracy =  tensor(85.6500)
Epoch 3/29
----------
train Loss: 0.4793 Acc: 77.3808 Auc: 0.6671 
val Loss: 0.3544 Acc: 86.1378 Auc: 0.7321 
new best accuracy =  tensor(86.1378)
Epoch 4/29
----------
train Loss: 0.4726 Acc: 77.8167 Auc: 0.6676 
val Loss: 0.3306 Acc: 87.2180 Auc: 0.7244 
new best accuracy =  tensor(87.2180)
Epoch 5/29
----------
train Loss: 0.4703 Acc: 78.0477 Auc: 0.6667 
val Loss: 0.3075 Acc: 88.4848 Auc: 0.7215 
new best accuracy =  tensor(88.4848)
Epoch 6/29
----------
train Loss: 0.4671 Acc: 78.3398 Auc: 0.6715 
val Loss: 0.3065 Acc: 88.8410 Au

In [9]:
model_ft = torch.nn.DataParallel(BCNN()).cuda()

#criterion = torch.nn.CrossEntropyLoss()
criterion = FocalLoss()

if use_gpu:
    criterion.cuda()
    model_ft = model_ft.cuda()

optimizer_ft = torch.optim.RMSprop(model_ft.parameters(), lr=0.0001)


# Run the functions and save the best model in the function model_ft.
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=30)

# Save model
#model_ft.save_state_dict('fine_tuned_best_model.pt')
torch.save(model_ft.state_dict())

Epoch 0/29
----------
LR is set to 0.001


D:\anaconda\lib\site-packages\ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


train Loss: 0.4373 Acc: 74.6363 Auc: 0.6482 
val Loss: 0.3439 Acc: 82.3653 Auc: 0.7006 
new best accuracy =  tensor(82.3653)
Epoch 1/29
----------
train Loss: 0.4138 Acc: 76.5494 Auc: 0.6616 
val Loss: 0.3173 Acc: 84.3809 Auc: 0.6992 
new best accuracy =  tensor(84.3809)
Epoch 2/29
----------
train Loss: 0.4080 Acc: 76.8502 Auc: 0.6607 
val Loss: 0.2894 Acc: 86.3732 Auc: 0.7201 
new best accuracy =  tensor(86.3732)
Epoch 3/29
----------
train Loss: 0.4025 Acc: 77.2298 Auc: 0.6657 
val Loss: 0.2832 Acc: 87.3000 Auc: 0.7276 
new best accuracy =  tensor(87.3000)
Epoch 4/29
----------


KeyboardInterrupt: 